In [93]:
from calculating_risk_for_single_community import calculating_risk_for_single_community as single_calculate
import numpy as np
import pandas as pd
import math
import datetime


In [62]:
population = pd.read_csv('processed_population.csv')

url_infe = "https://raw.githubusercontent.com/ANRGUSC/covid19_risk_estimation/master/data/Covid-19.csv"
infection = pd.read_csv(url_infe)

url_usc = "https://raw.githubusercontent.com/ANRGUSC/covid19_risk_estimation/master/data/Covid-19-R.csv"
risk_usc = pd.read_csv(url_usc)

url_rpi = "https://raw.githubusercontent.com/Yueyang-Li-Elfa/Risk-Score-RPI-Solver/master/Latest%20Risk%20Score/risk_score.csv"
risk_rpi = pd.read_csv(url_rpi)

url_umich = "https://raw.githubusercontent.com/skasralikar/Risk-Score-1-UMichZJU/master/data/output/LA_daily_out.csv"
risk_umich = pd.read_csv(url_umich,error_bad_lines=False)

# url_lmu = "https://raw.githubusercontent.com/wujj0326/COVID_Risk_Competition/master/Data/final_risk_score.csv"
# risk_lmu = pd.read_csv(url_lmu)

url_rmds3 = "https://raw.githubusercontent.com/sli525/rmds-lab-team3-project/master/risk_level.csv"
risk_rmds3 = pd.read_csv(url_rmds3)


In [63]:
'''
1) Daily new cases
2) Model outputs
    a) Aggregate RPI data to communities
3) Convert to Value between -1,0,1
4) Calculate correlation coefficient
'''
import warnings
warnings.filterwarnings('ignore')

In [64]:
# Change all regions/locations to Title Style
infection['Region'] = infection.apply(lambda x: x['Region'].replace('UNINCORPORATED - ','').title().replace('city of ',''), axis = 1)

print(population.head(),'\n',infection.head(),'\n',risk_usc.head(),'\n',risk_umich.head(),'\n',risk_rmds3.head())

            Region   Latitude   Longitude  Population
0       Northridge  34.234561 -118.536932       67755
1  Exposition Park  34.013654 -118.287211       45190
2          Norwalk  33.909280 -118.084917      104868
3        Hawthorne  33.918859 -118.348326      114032
4       El Segundo  33.917028 -118.415634       26864 
    Time Stamp         Region   Latitude   Longitude  Number of cases
0  03-16-2020       Alhambra  34.093042 -118.127060                2
1  03-16-2020        Arcadia  34.136208 -118.040150                1
2  03-16-2020  Beverly Hills  34.069650 -118.396306                1
3  03-16-2020  Boyle Heights  34.043689 -118.209768                5
4  03-16-2020         Carson  33.832204 -118.251755                1 
    Unnamed: 0  Time Stamp         Region   Latitude   Longitude  Risk-Score  \
0           0  03-16-2020       Alhambra  34.093042 -118.127060    0.158705   
1           1  03-16-2020        Arcadia  34.136208 -118.040150    0.142132   
2           2  03-16-

In [161]:
# Organize all model outputs to the format as ["Region","cases","risk","first_date","last_date"]
# USC
la_regions = risk_usc.Region.unique()

model_usc = pd.DataFrame()
model_usc["Region"] = la_regions
model_usc["cases"] = np.empty((len(model_usc), 0)).tolist()
model_usc["risk"] = np.empty((len(model_usc), 0)).tolist()
model_usc["first_date"] = np.empty((len(model_usc), 0)).tolist()
model_usc["last_date"] = np.empty((len(model_usc), 0)).tolist()

infection.sort_values(by="Time Stamp")
risk_usc.sort_values(by="Time Stamp")

for index, row in model_usc.iterrows():
    case_r = infection.loc[infection['Region'] == row["Region"]]
    risk_r = risk_usc.loc[risk_usc['Region'] == row["Region"]]
    model_usc["cases"][index]= case_r["Number of cases"].tolist()
    model_usc["risk"][index]= risk_r["Risk-Score"].tolist()
    
    # Get min and max Time stamp
    model_usc["first_date"][index]= [np.min(infection['Time Stamp']),np.min(risk_usc['Time Stamp'])]
    model_usc["last_date"][index]= [np.max(infection['Time Stamp']),np.max(risk_usc['Time Stamp'])]
    
print(model_usc)

                       Region  \
0                    Alhambra   
1                     Arcadia   
2               Beverly Hills   
3               Boyle Heights   
4                      Carson   
..                        ...   
237            South Pasadena   
238             Victoria Park   
239         Wellington Square   
240  West Whittier/Los Nietos   
241                Wilmington   

                                                 cases  \
0    [2, 1, 1, 2, 3, 3, 3, 4, 5, 6, 8, 8, 10, 10, 1...   
1    [1, 1, 2, 2, 2, 2, 2, 3, 4, 4, 6, 6, 8, 9, 9, ...   
2    [1, 1, 4, 4, 5, 7, 8, 12, 12, 13, 15, 21, 21, ...   
3    [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 7, 8, 8, 10, 11...   
4    [1, 1, 1, 1, 3, 3, 4, 6, 8, 9, 21, 26, 32, 35,...   
..                                                 ...   
237  [3, 5, 5, 6, 7, 12, 15, 15, 17, 17, 17, 20, 21...   
238  [0, 0, 0, 0, 0, 17, 17, 17, 17, 18, 18, 18, 18...   
239  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, ...   
240  [0, 0, 0, 0, 1, 10

In [172]:
# Organize all model outputs to the format as ["Region","cases","risk","first_date","last_date"]
# Umich
la_regions = risk_umich.Region.unique()

model_umich = pd.DataFrame()
model_umich["Region"] = la_regions
model_umich["cases"] = np.empty((len(model_umich), 0)).tolist()
model_umich["risk"] = np.empty((len(model_umich), 0)).tolist()
model_umich["first_date"] = np.empty((len(model_umich), 0)).tolist()
model_umich["last_date"] = np.empty((len(model_umich), 0)).tolist()

infection.sort_values(by="Time Stamp")
risk_umich.sort_values(by="Timestamp")

for index, row in model_umich.iterrows():
    case_r = infection.loc[infection['Region'] == row["Region"]]
    risk_r = risk_umich.loc[risk_umich['Region'] == row["Region"]]
    model_umich["cases"][index]= case_r["Number of cases"].tolist()
    model_umich["risk"][index]= risk_r["Risk_score"].tolist()
    
    # Get min and max Time stamp
    model_umich["first_date"][index]= [np.min(infection['Time Stamp']),np.min(risk_umich['Timestamp']).strip()]
    model_umich["last_date"][index]= [np.max(infection['Time Stamp']),np.max(risk_umich['Timestamp']).strip()]
    
print(model_umich)

              Region                                              cases  \
0              Acton  [0, 0, 0, 0, 0, 0, 5, 5, 5, 0, 0, 0, 0, 5, 5, ...   
1       Agoura Hills  [1, 3, 3, 5, 6, 9, 9, 9, 11, 10, 20, 20, 20, 2...   
2           Alhambra  [2, 1, 1, 2, 3, 3, 3, 4, 5, 6, 8, 8, 10, 10, 1...   
3           Altadena  [1, 2, 3, 3, 5, 5, 5, 6, 9, 10, 10, 10, 12, 14...   
4            Arcadia  [1, 1, 2, 2, 2, 2, 2, 3, 4, 4, 6, 6, 8, 9, 9, ...   
..               ...                                                ...   
152      Willowbrook  [1, 1, 1, 1, 1, 1, 2, 2, 5, 25, 27, 27, 27, 29...   
153       Wilmington  [7, 10, 10, 12, 13, 28, 32, 33, 33, 34, 40, 42...   
154  Wilshire Center  [1, 1, 2, 2, 4, 4, 5, 8, 8, 12, 40, 44, 46, 47...   
155         Winnetka  [1, 1, 1, 1, 2, 3, 5, 3, 8, 9, 9, 10, 11, 43, ...   
156   Woodland Hills  [2, 3, 3, 4, 5, 5, 7, 8, 11, 13, 14, 18, 19, 2...   

                                                  risk  \
0    [-1.5362257315534202, -1.79122101854

In [173]:
# Organize all model outputs to the format as ["Region","cases","risk","first_date","last_date"]
# RMDS3
la_regions = risk_rmds3.Region.unique()

model_rmds3 = pd.DataFrame()
model_rmds3["Region"] = la_regions
model_rmds3["cases"] = np.empty((len(model_rmds3), 0)).tolist()
model_rmds3["risk"] = np.empty((len(model_rmds3), 0)).tolist()
model_rmds3["first_date"] = np.empty((len(model_rmds3), 0)).tolist()
model_rmds3["last_date"] = np.empty((len(model_rmds3), 0)).tolist()

infection.sort_values(by="Time Stamp")
risk_rmds3.sort_values(by="Time Stamp")

for index, row in model_rmds3.iterrows():
    case_r = infection.loc[infection['Region'] == row["Region"]]
    risk_r = risk_rmds3.loc[risk_rmds3['Region'] == row["Region"]]
    model_rmds3["cases"][index]= case_r["Number of cases"].tolist()
    model_rmds3["risk"][index]= risk_r["risk_score"].tolist()
    
    # Get min and max Time stamp
    model_rmds3["first_date"][index]= [np.min(infection['Time Stamp']),np.min(risk_rmds3['Time Stamp']).strip()]
    model_rmds3["last_date"][index]= [np.max(infection['Time Stamp']),np.max(risk_rmds3['Time Stamp']).strip()]
    
print(model_rmds3)

               Region                                              cases  \
0        Agoura Hills  [1, 3, 3, 5, 6, 9, 9, 9, 11, 10, 20, 20, 20, 2...   
1            Alhambra  [2, 1, 1, 2, 3, 3, 3, 4, 5, 6, 8, 8, 10, 10, 1...   
2            Altadena  [1, 2, 3, 3, 5, 5, 5, 6, 9, 10, 10, 10, 12, 14...   
3             Arcadia  [1, 1, 2, 2, 2, 2, 2, 3, 4, 4, 6, 6, 8, 9, 9, ...   
4              Arleta  [1, 2, 2, 4, 5, 5, 7, 7, 23, 24, 24, 26, 29, 3...   
..                ...                                                ...   
151             Acton  [0, 0, 0, 0, 0, 0, 5, 5, 5, 0, 0, 0, 0, 5, 5, ...   
152  Lake Los Angeles  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 7, 8, 8, ...   
153    South El Monte  [1, 1, 1, 1, 1, 1, 3, 4, 0, 0, 0, 0, 0, 5, 7, ...   
154           Bassett  [0, 0, 0, 0, 0, 0, 6, 6, 6, 6, 13, 15, 15, 20,...   
155            Avalon  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...   

                                                  risk  \
0    [-0.399918983, -0.179499

In [ ]:
# Organize all model outputs to the format as ["Region","cases","risk","first_date","last_date"]
# RPI

la_regions = risk_rpi["community"].unique()
print(la_regions)
print(risk_rpi.dropna())
risk_rpi = risk_rpi.dropna(thresh=26)
comm_rpi = risk_rpi.groupby(["community","weekday"])["risk_score"].apply(lambda x : x.astype(float).mean()).reset_index()

la_regions = comm_rpi["community"].unique()
print(la_regions)


model_rpi = pd.DataFrame()
model_rpi["Region"] = la_regions
model_rpi["cases"] = np.empty((len(model_rpi), 0)).tolist()
model_rpi["risk"] = np.empty((len(model_rpi), 0)).tolist()
model_rpi["first_date"] = np.empty((len(model_rpi), 0)).tolist()
model_rpi["last_date"] = np.empty((len(model_rpi), 0)).tolist()

infection.sort_values(by="Time Stamp")
risk_rpi.sort_values(by="Time Stamp")

for index, row in model_rpi.iterrows():
    case_r = infection.loc[infection['Region'] == row["Region"]]
    risk_r = risk_rpi.loc[risk_rpi['Region'] == row["Region"]]
    model_rpi["cases"][index]= case_r["Number of cases"].tolist()
    model_rpi["risk"][index]= risk_r["risk_score"].tolist()
    
    # Get min and max Time stamp
    model_rpi["first_date"][index]= [np.min(infection['Time Stamp']),np.min(risk_rpi['Time Stamp'])]
    model_rpi["last_date"][index]= [np.max(infection['Time Stamp']),np.max(risk_rpi['Time Stamp'])]
    
print(model_rpi)

In [ ]:
# Error Calculation Function

def cal_error(n_ahead, case, risk):
    n = len(case)
    new_infe = []
    risk_infe = []
    for i in range(n-n_ahead):
        new_infe.append(case[i+n_ahead]-case[i])
        risk_infe.append(risk[i])
    new_infe = (np.array(new_infe)-np.mean(new_infe)) / np.std(new_infe)
    risk_infe = (np.array(risk_infe)-np.mean(risk_infe)) / np.std(risk_infe)
    # desc_var(new_infe,risk_infe)
    return np.corrcoef(new_infe, risk_infe)[0,1]

In [194]:
# Choose intersected data
def column_slice(df,data_column,last_date_column,case_risk,first_d,last_d):
    case_len = (first_d - last_d).days
    format_date = ["%m-%d-%Y","%Y-%m-%d"]
    case_l_delta = (last_d - datetime.datetime.strptime(df[last_date_column][0][case_risk],format_date[case_risk])).days
    case_m = df[data_column].tolist()

    for i in range(len(case_m)):
        j = len(case_m[i])
        if case_l_delta < 0:
            case_m[i] = case_m[i][(case_l_delta+case_len):case_l_delta]
        elif case_l_delta == 0:
            case_m[i] = case_m[i][case_len:]

    df[data_column] = case_m

In [248]:
# Ensemble

# merge two or more dataframes into one

df_ensemble = model_usc.merge(model_umich,on='Region').merge(model_rmds3,on='Region')
df_ensemble = df_ensemble[["Region","cases","risk_x","risk_y","risk","first_date_x","last_date_x","first_date_y","last_date_y","first_date","last_date"]]
df_ensemble = df_ensemble.rename(columns={'risk_x': 'risk_usc','risk_y':'risk_umich','risk':'risk_rmds3','first_date_x':'first_date_usc','first_date_y':'first_date_umich','first_date':'first_date_rmds3','last_date_x':'last_date_usc','last_date_y':'last_date_umich','last_date':'last_date_rmds3'})

'''
Choose the intersection
4-7-2020,7-26-2020
'''
first_d = datetime.datetime.strptime("4-7-2020","%m-%d-%Y")
last_d = datetime.datetime.strptime("7-26-2020","%m-%d-%Y")

# cases portion
column_slice(df_ensemble,"cases","last_date_usc",0,first_d,last_d)
# usc portion
column_slice(df_ensemble,"risk_usc","last_date_usc",0,first_d,last_d)
# umich portion
column_slice(df_ensemble,"risk_umich","last_date_umich",1,first_d,last_d)
# rmdse portion
column_slice(df_ensemble,"risk_rmds3","last_date_umich",1,first_d,last_d)

print(len(df_ensemble['risk_usc'][100]),len(df_ensemble['cases'][100]),len(df_ensemble['risk_umich'][0]),len(df_ensemble['risk_rmds3'][0]))

df_ensemble = df_ensemble[df_ensemble['risk_rmds3'].str.len()>109]
df_ensemble = df_ensemble[df_ensemble['risk_umich'].str.len()>109]

df_ensemble = df_ensemble[['Region','cases','risk_usc','risk_umich','risk_rmds3']]
df_ensemble["first_last"] = [["4-7-2020","7-26-2020"] for i in df_ensemble.index]
df_ensemble

110 110 110 110


,Region,cases,risk_usc,risk_umich,risk_rmds3,first_last
0,Alhambra,"[4, 5, 6, 8, 8, 10, 10, 11, 13, 13, 32, 33, 33...","[0.5324290755863141, 0.6853968195688455, 0.778...","[0.106636100882027, 0.2265174256780509, 0.2085...","[0.07545186268, 0.1071458062, 0.03452694789, 0...","[4-7-2020, 7-26-2020]"
1,Arcadia,"[4, 4, 6, 6, 8, 9, 9, 10, 12, 19, 20, 22, 22, ...","[0.8012437032575108, 0.8429061305300106, 0.766...","[0.05414417446446471, 0.10236703478934976, -0....","[0.08400861621, -0.2401427699, -0.2141583822, ...","[4-7-2020, 7-26-2020]"
2,Beverly Hills,"[71, 73, 74, 74, 76, 78, 78, 78, 80, 80, 45, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.451220534069038, 0.7637192169841681, 1.1094...","[0.5957496667000001, 0.3991195659, 0.110809243...","[4-7-2020, 7-26-2020]"
3,Boyle Heights,"[66, 65, 69, 73, 20, 97, 116, 129, 134, 148, 1...","[2.7047124068527792, 4.958781813735855, 7.4185...","[0.182772223548144, 0.4177911607818887, 0.4080...","[0.3467186173, 0.4664587684000001, 0.387939178...","[4-7-2020, 7-26-2020]"
4,Carson,"[174, 182, 188, 191, 66, 45, 49, 58, 257, 264,...","[0.6591815133421615, 1.0922772161063872, 1.784...","[0.5471773616917418, 0.7994671847909767, 0.865...","[0.7849067956, 0.9356088288, 1.018973142, 0.83...","[4-7-2020, 7-26-2020]"
...,...,...,...,...,...,...
150,Downtown,"[27, 28, 31, 31, 15, 34, 36, 41, 49, 49, 49, 5...","[0.9516842852448966, 1.4249682732461564, 1.790...","[0.056884691930895036, 0.11896033223524845, -0...","[0.1544235442, 0.2683652419, 0.342485476599999...","[4-7-2020, 7-26-2020]"
152,Koreatown,"[61, 62, 68, 71, 21, 78, 93, 104, 108, 113, 11...","[2.226665231401061, 2.9480208621527426, 3.9420...","[0.20704992931972965, 0.29071757057264863, 0.3...","[0.0919620158, 0.07333970043, 0.2948804994, 0....","[4-7-2020, 7-26-2020]"
153,Mt. Washington,"[16, 16, 16, 17, 0, 22, 25, 31, 32, 32, 33, 33...","[0.4281123462355465, 0.9139535957630532, 1.603...","[-0.7727920147400095, -0.8282857747440592, -0....","[-0.01789836622, -0.6995464855, -0.9768389254,...","[4-7-2020, 7-26-2020]"
154,South Pasadena,"[20, 21, 26, 28, 7, 30, 34, 38, 51, 56, 59, 60...","[1.5002113718388437, 1.6878883811683054, 1.842...","[-0.23949868795886745, -0.26018694193378983, -...","[-0.1849749572, -0.3275514723, -0.4784374557, ...","[4-7-2020, 7-26-2020]"


In [249]:
df_ensemble.dropna()
df_ensemble.loc[df_ensemble['Region']=='Santa Clarita']

,Region,cases,risk_usc,risk_umich,risk_rmds3,first_last
41,Santa Clarita,"[178, 188, 196, 201, 65, 221, 244, 273, 283, 2...","[-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....","[0.4623781927794273, 0.7045796511304945, 0.759...","[0.6723224337, 0.7616595851000001, 0.645837100...","[4-7-2020, 7-26-2020]"


In [278]:
def n_case(n_ahead, case):
    n = len(case)
    new_infe = []
    for i in range(n):
        if i<n_ahead:
            new_infe.append(0)
        else:
            new_infe.append(case[i]-case[i-n_ahead])
    new_infe = (np.array(new_infe)-np.min(new_infe)) / (np.max(new_infe)-np.min(new_infe))
    new_infe = np.round(new_infe,6)
    
    return new_infe.tolist()

def n_risk(risk):
    n = len(risk)

    risk = (np.array(risk)-np.min(risk)) / (np.max(risk)-np.min(risk))
    risk = np.round(risk,6)
    return risk.tolist()

In [280]:
df_ensemble['3-case'] = 0
df_ensemble['3-case'] = df_ensemble['3-case'].astype('object')
df_ensemble['7-case'] = 0
df_ensemble['7-case'] = df_ensemble['7-case'].astype('object')
df_ensemble['n-usc'] = 0
df_ensemble['n-usc'] = df_ensemble['n-usc'].astype('object')
df_ensemble['n-umich'] = 0
df_ensemble['n-umich'] = df_ensemble['n-umich'].astype('object')
df_ensemble['n-rmds3'] = 0
df_ensemble['n-rmds3'] = df_ensemble['n-rmds3'].astype('object')

for index, row in df_ensemble.iterrows():
    case = np.array(row['cases'], dtype = 'float')
    risk_usc = np.array(row['risk_usc'], dtype = 'float')
    risk_umich = np.array(row['risk_umich'], dtype = 'float')
    risk_rmds3 = np.array(row['risk_rmds3'], dtype = 'float')
    
    if sum(risk_usc)<=0:
        continue
    df_ensemble.set_value(index, '3-case', n_case(3, case))
    #df_ensemble.set_value(index, '7-case', n_case(7, case))
    df_ensemble.set_value(index, 'n-usc', n_risk(risk_usc))
    df_ensemble.set_value(index, 'n-umich', n_risk(risk_umich))
    df_ensemble.set_value(index, 'n-rmds3', n_risk(risk_rmds3))

338.0
120.0
271.0
1192.0
487.0
171.0
127.0
304.0
630.0
447.0
251.0
625.0
256.0
770.0
114.0
401.0
292.0
155.0
703.0
180.0
220.0
189.0
309.0
77.0
199.0
114.0
128.0
562.0
112.0
140.0
137.0
139.0
178.0
279.0
70.0
884.0
83.0
220.0
710.0
431.0
226.0
631.0
82.0
41.0
275.0
54.0
1474.0
600.0
42.0
1721.0
46.0
60.0
634.0
216.0
71.0
132.0
64.0
115.0
54.0
76.0
176.0
111.0
536.0
217.0
84.0
665.0
36.0
92.0
186.0
47.0
140.0
196.0
564.0
251.0
81.0
70.0
151.0
653.0
307.0
92.0
504.0
810.0
910.0
623.0
681.0
340.0
141.0
42.0
675.0
72.0
1310.0
38.0
204.0
308.0
208.0
279.0
784.0
527.0
275.0
58.0
102.0
1240.0
133.0
279.0
471.0
915.0
602.0
66.0
139.0
849.0
498.0
300.0
534.0
140.0
552.0
1360.0
433.0
496.0
548.0
364.0
123.0
310.0
127.0
87.0
354.0
199.0
355.0
654.0
612.0
1240.0
365.0
1886.0
169.0
121.0
152.0
78.0
210.0
229.0
115.0
89.0
430.0


In [252]:
df_ensemble

,Region,cases,risk_usc,risk_umich,risk_rmds3,first_last,3-case,7-case,n-usc,n-umich,n-rmds3
0,Alhambra,"[4, 5, 6, 8, 8, 10, 10, 11, 13, 13, 32, 33, 33...","[0.5324290755863141, 0.6853968195688455, 0.778...","[0.106636100882027, 0.2265174256780509, 0.2085...","[0.07545186268, 0.1071458062, 0.03452694789, 0...","[4-7-2020, 7-26-2020]","[0.495522, 0.495522, 0.495522, 0.501493, 0.5, ...","[0.434659, 0.434659, 0.434659, 0.434659, 0.434...","[0.029443, 0.037902, 0.043071, 0.042368, 0.037...","[0.076882, 0.124125, 0.11706, 0.140462, 0.1059...","[0.014126, 0.025066, 0.0, 0.031185, 0.011135, ..."
1,Arcadia,"[4, 4, 6, 6, 8, 9, 9, 10, 12, 19, 20, 22, 22, ...","[0.8012437032575108, 0.8429061305300106, 0.766...","[0.05414417446446471, 0.10236703478934976, -0....","[0.08400861621, -0.2401427699, -0.2141583822, ...","[4-7-2020, 7-26-2020]","[0.493671, 0.493671, 0.493671, 0.50211, 0.5105...","[0.435146, 0.435146, 0.435146, 0.435146, 0.435...","[0.09859, 0.103717, 0.094347, 0.075194, 0.0550...","[0.233678, 0.263831, 0.167441, 0.215688, 0.211...","[0.32076, 0.200434, 0.210079, 0.084084, 0.2442..."
2,Beverly Hills,"[71, 73, 74, 74, 76, 78, 78, 78, 80, 80, 45, 1...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.451220534069038, 0.7637192169841681, 1.1094...","[0.5957496667000001, 0.3991195659, 0.110809243...","[4-7-2020, 7-26-2020]","[0.495345, 0.495345, 0.495345, 0.500931, 0.500...","[0.47698, 0.47698, 0.47698, 0.47698, 0.47698, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.146946, 0.200435, 0.259605, 0.304837, 0.196...","[0.193656, 0.158177, 0.106155, 0.064906, 0.080..."
3,Boyle Heights,"[66, 65, 69, 73, 20, 97, 116, 129, 134, 148, 1...","[2.7047124068527792, 4.958781813735855, 7.4185...","[0.182772223548144, 0.4177911607818887, 0.4080...","[0.3467186173, 0.4664587684000001, 0.387939178...","[4-7-2020, 7-26-2020]","[0.483088, 0.483088, 0.483088, 0.486123, 0.463...","[0.415947, 0.415947, 0.415947, 0.415947, 0.415...","[0.08267, 0.151566, 0.22675, 0.273886, 0.09207...","[0.0, 0.032714, 0.031357, 0.035227, 0.033165, ...","[0.01363, 0.031131, 0.019655, 0.007498, 0.0170..."
4,Carson,"[174, 182, 188, 191, 66, 45, 49, 58, 257, 264,...","[0.6591815133421615, 1.0922772161063872, 1.784...","[0.5471773616917418, 0.7994671847909767, 0.865...","[0.7849067956, 0.9356088288, 1.018973142, 0.83...","[4-7-2020, 7-26-2020]","[0.537512, 0.537512, 0.537512, 0.553656, 0.427...","[0.477748, 0.477748, 0.477748, 0.477748, 0.477...","[0.215313, 0.356777, 0.582921, 0.831433, 0.439...","[0.0469, 0.139671, 0.163905, 0.285596, 0.24244...","[0.187903, 0.245497, 0.277356, 0.206181, 0.185..."
...,...,...,...,...,...,...,...,...,...,...,...
150,Downtown,"[27, 28, 31, 31, 15, 34, 36, 41, 49, 49, 49, 5...","[0.9516842852448966, 1.4249682732461564, 1.790...","[0.056884691930895036, 0.11896033223524845, -0...","[0.1544235442, 0.2683652419, 0.342485476599999...","[4-7-2020, 7-26-2020]","[0.487805, 0.487805, 0.487805, 0.497561, 0.456...","[0.420091, 0.420091, 0.420091, 0.420091, 0.420...","[0.157465, 0.235774, 0.296288, 0.343286, 0.0, ...","[0.025068, 0.036294, 0.001694, 0.013317, 0.0, ...","[0.02041, 0.036658, 0.047228, 0.01186, 0.02055..."
152,Koreatown,"[61, 62, 68, 71, 21, 78, 93, 104, 108, 113, 11...","[2.226665231401061, 2.9480208621527426, 3.9420...","[0.20704992931972965, 0.29071757057264863, 0.3...","[0.0919620158, 0.07333970043, 0.2948804994, 0....","[4-7-2020, 7-26-2020]","[0.539235, 0.539235, 0.539235, 0.559356, 0.456...","[0.473684, 0.473684, 0.473684, 0.473684, 0.473...","[0.286286, 0.379032, 0.506835, 0.605607, 0.160...","[0.116034, 0.18903, 0.21637, 0.240665, 0.21719...","[0.015169, 0.0, 0.180452, 0.294412, 0.232098, ..."
153,Mt. Washington,"[16, 16, 16, 17, 0, 22, 25, 31, 32, 32, 33, 33...","[0.4281123462355465, 0.9139535957630532, 1.603...","[-0.7727920147400095, -0.8282857747440592, -0....","[-0.01789836622, -0.6995464855, -0.9768389254,...","[4-7-2020, 7-26-2020]","[0.564394, 0.564394, 0.564394, 0.568182, 0.503...","[0.481481, 0.481481, 0.

In [263]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

def lin_reg(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=9)
    lin_reg_mod = LinearRegression()
    lin_reg_mod.fit(X_train, y_train)
    pred = lin_reg_mod.predict(X_test)
    # RMSE
    test_set_rmse = (np.sqrt(mean_squared_error(y_test, pred)))
    # r square
    test_set_r2 = lin_reg_mod.score(X_test,y_test)

    output = np.array([lin_reg_mod.coef_,lin_reg_mod.intercept_,test_set_rmse])
    return output

In [264]:
df_ensemble = df_ensemble[df_ensemble["3-case"] != 0]
df_ensemble.to_csv("ensemble_pre.csv")

In [265]:
df_ensemble['ensemble'] = 0
df_ensemble['ensemble'] = df_ensemble['ensemble'].astype('object')

# Ensemble to do 3-day prediction
for index, row in df_ensemble.iterrows():
    try:


        y = np.array(row['3-case'], dtype = 'object')
        x_usc = np.array(row['n-usc'], dtype = 'object')
        x_umich = np.array(row['n-umich'], dtype = 'object')
        x_rmds3 = np.array(row['n-rmds3'], dtype = 'object')
        X = pd.DataFrame({'n_usc': x_usc, 'n_umich': x_umich,'n_rmds3': x_rmds3})
        df_ensemble.set_value(index, 'ensemble', lin_reg(X,y))
    except:
        pass

In [266]:
df_ensemble.to_csv("ensemble_result.csv")

In [268]:
df_ensemble['predict'] = 0
df_ensemble['predict'] = df_ensemble['predict'].astype('object')
for index, row in df_ensemble.iterrows():
    try:
        df_ensemble.set_value(index, 'predict', row["n-usc"][-1]*row["ensemble"][0][0]+row["n-umich"][-1]*row["ensemble"][0][1]+row["n-rmds3"][-1]*row["ensemble"][0][2]+row["ensemble"][1])
    except:
        pass
df_ensemble.to_csv("ensemble_prediction.csv")